In [ ]:
# pip install pandas

In [ ]:
import pandas as pd
import re

# 1. Load the raw data
df = pd.read_csv('FS-data-80475.csv')

# 2. Impute Missing KPI Names (Handles Nulls)
print(f"Rows with missing 'english_name' before: {df['english_name'].isnull().sum()}")
lookup_map = df.dropna(subset=['account_id', 'english_name']) \
               .drop_duplicates(subset=['account_id']) \
               .set_index('account_id')['english_name'] \
               .to_dict()
df['english_name'] = df['english_name'].fillna(df['account_id'].map(lookup_map))
df.dropna(subset=['english_name'], inplace=True)
print(f"Rows with missing 'english_name' after imputing: {df['english_name'].isnull().sum()}")

# 3. Clean and Standardize All KPI Names (Fixes Duplicates & Special Chars)
def clean_kpi_name(name):
    name = re.sub(r'[\(\)\[\]\{\}\n]', '', name) # Remove special characters
    name = re.sub(r'\s+', ' ', name)            # Replace multiple spaces with one
    name = name.strip()                         # Remove leading/trailing whitespace
    return name
df['english_name'] = df['english_name'].apply(clean_kpi_name)
print("\nKPI names have been cleaned and standardized.") 

# 4. Create a Datetime Index
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-01')

# 5. Pivot the Data (Now with Clean Columns)
time_series_df = df.pivot_table(
    index='date',
    columns='english_name',
    values='monthly_value',
    aggfunc='sum' # Use 'sum' to combine values from cleaned duplicate names
)

# --- NEW STEP: Remove Columns Containing Only Zeros ---
print(f"\nOriginal number of KPIs (columns): {time_series_df.shape[1]}")

# Identify columns where ALL values are zero
cols_to_remove = time_series_df.columns[(time_series_df == 0).all()]

# Drop these columns from the DataFrame
time_series_df = time_series_df.drop(columns=cols_to_remove)

print(f"Removed {len(cols_to_remove)} columns that only contained zeros.")
print(f"New final number of KPIs (columns): {time_series_df.shape[1]}")
# --- END OF NEW STEP ---


# 6. Save the final cleaned dataset to your system
cleaned_file_path = 'cleaned_model_ready_data.csv'
time_series_df.to_csv(cleaned_file_path)

print(f"\n✅ Cleaned data (with all-zero columns removed) has been saved to '{cleaned_file_path}'")
print("✅ Data is now ready for model training.")
time_series_df.head()

In [ ]:
# !pip install statsmodels matplotlib

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Select the KPI you want to forecast. Let's use 'TOTAL NEW VEHICLE DEPT'.
kpi_to_forecast = 'TOTAL NEW VEHICLE DEPT'

# Get the data for just this KPI from your cleaned DataFrame.
# .fillna(0) is a simple way to handle any missing months for this KPI.
kpi_series = time_series_df[kpi_to_forecast].fillna(0)

# Plot the historical data to see the pattern
plt.figure(figsize=(12, 6))
plt.plot(kpi_series)
plt.title(f'Historical Monthly Values for {kpi_to_forecast}')
plt.xlabel('Date')
plt.ylabel('Monthly Value')
plt.grid(True)
plt.show()

In [ ]:
# --- Corrected Forecasting Loop for All KPIs ---

import statsmodels.api as sm

# Create an empty DataFrame to store all our forecasts
all_forecasts = pd.DataFrame()
kpi_columns = time_series_df.columns
print(f"Starting forecast for {len(kpi_columns)} KPIs...")

# Loop through each KPI column
for kpi in kpi_columns:
    kpi_series = time_series_df[kpi].fillna(0)
    
    try:
        model = sm.tsa.statespace.SARIMAX(kpi_series,
                                        order=(1, 1, 1),
                                        seasonal_order=(1, 1, 1, 12))
        results = model.fit(disp=False)
        
        # This is the corrected part that gets 3 months
        forecast = results.get_forecast(steps=3)
        predicted_values = forecast.predicted_mean
        
        all_forecasts[kpi] = predicted_values
        
    except Exception as e:
        all_forecasts[kpi] = [0, 0, 0]

print("\n\n✅ All KPIs have been forecasted for 3 months!")

# Save the final forecast results
forecast_file_path = '3_month_forecast_all_kpis.csv'
all_forecasts.to_csv(forecast_file_path)
print(f"✅ Forecast has been saved to '{forecast_file_path}'")

# Display the final forecast to confirm it has 3 rows
all_forecasts.head()

In [ ]:
print("--- Forecasted Values for Jan, Feb, Mar 2025 ---")
all_forecasts.head()



In [ ]:
correlation_matrix = time_series_df.corr()

print("✅ Correlation matrix calculated successfully.")

# Display the top 5 rows and columns of the matrix
correlation_matrix.head()

# forecast_file_path = '3_month_forecast_all_kpis.csv'

# # Use the .to_csv() function to save the DataFrame
# all_forecasts.to_csv(forecast_file_path)

In [ ]:
# !pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org seaborn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the size of the plot
plt.figure(figsize=(12, 10))

# Create the heatmap using seaborn
sns.heatmap(correlation_matrix, cmap='coolwarm') # 'coolwarm' is a good color map for correlations

plt.title('Correlation Matrix of All KPIs')
plt.show()

In [ ]:
# Define a file path for your correlation matrix
correlation_file_path = 'cleaned_correlation_matrix.csv'

# Use the .to_csv() function to save the DataFrame
correlation_matrix.to_csv(correlation_file_path)

print(f"✅ Successfully saved the correlation matrix to: {correlation_file_path}")

In [ ]:
# # --- FINAL STEP: Model Accuracy Calculation using Train-Test Split ---

# import pandas as pd
# import numpy as np
# import statsmodels.api as sm
# from tqdm import tqdm # A library to show a progress bar

# # 1. LOAD YOUR FINAL CLEANED DATA
# time_series_df = pd.read_csv('cleaned_model_ready_data.csv', index_col='date', parse_dates=True)

# # 2. CHRONOLOGICAL TRAIN-TEST SPLIT
# print("--- Splitting data into Training and Testing sets ---")
# cutoff_date = '2024-01-01'
# train_data = time_series_df[time_series_df.index < cutoff_date]
# test_data = time_series_df[time_series_df.index >= cutoff_date]
# print(f"Training data: {train_data.shape[0]} months, Testing data: {test_data.shape[0]} months")

# # 3. TRAIN ON TRAIN SET, FORECAST FOR TEST SET DURATION
# print("\n--- Training models and forecasting to get accuracy score... ---")
# # This may take a few minutes to run.
# forecasts = pd.DataFrame()
# for kpi in tqdm(train_data.columns, desc="Forecasting KPIs"):
#     train_series = train_data[kpi].fillna(0)
#     try:
#         model = sm.tsa.statespace.SARIMAX(train_series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
#         results = model.fit(disp=False)
#         prediction = results.get_forecast(steps=len(test_data))
#         forecasts[kpi] = prediction.predicted_mean
#     except Exception as e:
#         forecasts[kpi] = [0] * len(test_data)

# # 4. CALCULATE ACCURACY (MAPE)
# print("\n--- Calculating final model accuracy ---")
# actuals = test_data[forecasts.columns]
# percentage_error = np.abs((actuals - forecasts) / actuals) * 100
# percentage_error.replace([np.inf, -np.inf], np.nan, inplace=True)
# mape = np.nanmean(percentage_error.values)

# # 5. DISPLAY THE FINAL RESULT
# print("\n" + "="*50)
# print(f"✅ The Model's Mean Absolute Percentage Error (MAPE) is: {mape:.2f}%")
# print("="*50)

In [ ]:
# # --- FINAL ACCURACY: Calculate MAPE on Top 10 KPIs ---

# # 1. Find the 10 KPIs with the highest average historical value
# top_10_kpis = time_series_df.mean().nlargest(10).index.tolist()

# print("--- Calculating a more meaningful MAPE on the Top 10 KPIs ---")
# print("Top 10 KPIs by average value:")
# for kpi in top_10_kpis:
#     print(f"  - {kpi}")

# # 2. Filter your actuals and forecasts to only include these top KPIs
# actuals_top_10 = test_data[top_10_kpis]
# forecasts_top_10 = forecasts[top_10_kpis]

# # 3. Recalculate the MAPE
# percentage_error_top_10 = np.abs((actuals_top_10 - forecasts_top_10) / actuals_top_10) * 100
# percentage_error_top_10.replace([np.inf, -np.inf], np.nan, inplace=True)
# mape_top_10 = np.nanmean(percentage_error_top_10.values)

# # 4. DISPLAY THE FINAL, MEANINGFUL RESULT
# print("\n" + "="*50)
# print(f"✅ The MAPE for the Top 10 most significant KPIs is: {mape_top_10:.2f}%")
# print("="*50)